In [1]:
import numpy as np
import pyci
import psi4

In [3]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('6-31g*', '../.xyz/licn.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
aoint.save_ao_dpints()
aoint.save_ao_qdints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
ao_dipoles = np.load('ao_dpints.npz')
ao_quadrupoles = np.load('ao_qdints.npz')
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
mo_dpx = aoint.matrix_ao2mo(Ca, ao_dipoles['dpx_aoints'])
mo_dpy = aoint.matrix_ao2mo(Ca, ao_dipoles['dpy_aoints'])
mo_dpz = aoint.matrix_ao2mo(Ca, ao_dipoles['dpz_aoints'])
mo_qdxx = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdxx_aoints'])
mo_qdyy = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdyy_aoints'])
mo_qdzz = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdzz_aoints'])
del ao_erints, ao_dipoles, ao_quadrupoles

Size of the ERI tensor will be 0.03 Gb

Ground state SCF Energy : -99.77249855 

Nuclear repulsion energy : 27.84883421 

Total electronic energy : -127.62133275 



In [4]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 45  
Molecular orbitals: 45  
Spin Orbitals     : 90  
Alpha Orbitals    : 8  
Beta Orbitals     : 8  
Occupied Orbitals : 8  
Virtual Orbitals  : 37  



In [5]:
scf_energy = aoint.scf_energy
psi4_cisd = psi4.energy('CISD')

In [6]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
singles = True
full_cis = True
doubles = False
doubles_iiaa = True
doubles_iiab = True
doubles_ijaa = True
doubles_ijab_A = True
doubles_ijab_B = True
options = [singles, full_cis, doubles,
           doubles_iiaa, doubles_iiab,doubles_ijaa,
           doubles_ijab_A, doubles_ijab_B]
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)


([1, 296, 0, 0, 0, 0, 0], 297)

In [7]:
type(csfs)

list

In [8]:
HCISD = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=2)

In [9]:
HCISD.shape

(297, 297)

In [10]:
vals, vecs = np.linalg.eigh(HCISD)

In [11]:
# Comparing pyci with psi4
print("pyci CISD E0: {e:16.16f}".format(e=vals[0]))
print("psi4 CISD E0: {e:16.16f}".format(e=psi4_cisd))
print("dE : {dE:1.2E}".format(dE=abs(psi4_cisd-vals[0])))


pyci CISD E0: -99.7724985466440870
psi4 CISD E0: -100.0498421654397561
dE : 2.77E-01


In [12]:
csf_dpx  = pyci.configint.rcisd.comp_oeprop_matrix(mo_dpx, orbinfo, active_space, options)
csf_dpy  = pyci.configint.rcisd.comp_oeprop_matrix(mo_dpy, orbinfo, active_space, options)
csf_dpz  = pyci.configint.rcisd.comp_oeprop_matrix(mo_dpz, orbinfo, active_space, options)
csf_qdxx  = pyci.configint.rcisd.comp_oeprop_matrix(mo_qdxx, orbinfo, active_space, options)
csf_qdyy  = pyci.configint.rcisd.comp_oeprop_matrix(mo_qdyy, orbinfo, active_space, options)
csf_qdzz  = pyci.configint.rcisd.comp_oeprop_matrix(mo_qdzz, orbinfo, active_space, options)

In [15]:
from pyci.integrators import RK4

In [16]:
from pyci.utils import excite

ImportError: cannot import name 'excite' from 'pyci.utils' (/home/vijay/.conda/envs/p4env/lib/python3.10/site-packages/pyci/utils/__init__.py)

In [ ]:
y0 = vec[0]
t0 = 0.0
tf = 20.0
dt = 0.0001
sigma

propagator = RK4()